In [2]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda
import requests
import pandas as pd
import numpy as np

# Pair Programming ETL I
Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

In [ ]:
url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'

En este caso os recomendamos que uséis el producto meteo para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el dataset.


Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

Requisitos de este ejercicio de pair programming:

- Deberéis meter toda la información climática en un único dataframe.
- Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
- Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
- Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:
- Crearos un diccionario donde:
    -   Las keys sean los nombres de los paises
    -   Los values sean las coordenadas de los paises en formato lista.

- Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura:
for key, value in diccionario.items():
    print(key, value)

- Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.
Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [3]:
# Creamos un diccionario con la latitud y longitud de los paises:                         
                        # Pais : [Latitud, Longitud]
paises_coordenadas = {"USA": [39.7837304, -100.445882], 
                        "Australia": [-24.7761086, 134.755], 
                        "South Africa": [-28.8166236, 24.991639], 
                        "New Zealand": [-41.5000831, 172.8344077], 
                        "Papua New Guinea": [-5.6816069, 144.2489081]}

In [9]:
def llamada_API(dicc, producto, datafr):
#Iteramos por el diccionario en busca de nuestros valores
    for k, v in dicc.items():
        lat = v[0]
        lon = v[1]
    
    # hacemos la llamada  a la API
        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
    # Ponemos esto para comprobar si todo está correcto.    
        if codigo_estado == 200:
            print('La peticion de se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usuario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        
     # convertimos los resultados en un dataframe:
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    # Creamos una nueva columna con el nombre del pais para poder hacer la union.
        df["Pais"] = k
    #hacemos la union con el df anterior
        df_final = pd.merge(df , datafr, on='Pais', how = "left")
    
    return df_final

In [10]:
#con el diccionario que teniamos creamos un df y le resetemos el indice
df_paises_tiempo = pd.DataFrame.from_dict(paises_coordenadas).T.reset_index()
df_paises_tiempo.rename(columns = {"index" : "Pais", 0: "Latitud", 1: "Longitud"}, inplace= True)
df_paises_tiempo

,Pais,Latitud,Longitud
0,USA,39.783730,-100.445882
1,Australia,-24.776109,134.755000
2,South Africa,-28.816624,24.991639
3,New Zealand,-41.500083,172.834408
4,Papua New Guinea,-5.681607,144.248908


In [16]:
df = llamada_API( paises_coordenadas, 'meteo', df_paises_tiempo)
df.sample(2)

La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,Pais,Latitud,Longitud
24,75,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 80, 'speed': ...",26,2,12,1008,rain,2,0,90,3,Papua New Guinea,-5.681607,144.248908
26,81,4,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,2,12,1010,none,2,0,80,3,Papua New Guinea,-5.681607,144.248908


In [15]:
df.isnull().sum()

timepoint            0
cloudcover           0
highcloud            0
midcloud             0
lowcloud             0
rh_profile           0
wind_profile         0
temp2m               0
lifted_index         0
rh2m                 0
msl_pressure         0
prec_type            0
prec_amount          0
snow_depth           0
wind10m.direction    0
wind10m.speed        0
Pais                 0
Latitud              0
Longitud             0
dtype: int64

In [17]:
df.to_csv("../data/00-datos_clima.csv")

In [18]:
df.to_pickle('../data/00-datos-clima.pkl')